In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import json
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
import pathlib
from statsmodels.stats.multitest import multipletests

In [2]:
or_cutoff = 1.6
neg_lgp_cutoff = 2
mask_quantile_to_max = 0.8

gene_cluster = '0'

In [3]:
# Parameters
gene_cluster = "13"
or_cutoff = 1.6
neg_lgp_cutoff = 3
mask_quantile_to_max = 0.8


In [4]:
output_dir = 'MotifEnrichment'
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
with open('GeneCluster.relatedDMR.index.json') as f:
    use_dmrs = json.load(f)[gene_cluster]

In [6]:
gene_clusters = anndata.read_h5ad('GeneClustering.h5ad').obs['leiden']
use_genes = gene_clusters[gene_clusters == gene_cluster].index

In [7]:
print(use_genes.size, 'genes in gene cluster', gene_cluster)
print(len(use_dmrs), 'related DMRs')

24 genes in gene cluster 13
4059 related DMRs


In [8]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5') as hdf:
    dmr_bed_df = hdf['bed'].loc[use_dmrs].copy()
dmr_bed_df.shape

(4059, 4)

In [9]:
dmr_annot = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/MotifScan.h5ad'
)
# mask small motif scores
motif_cutoff = pd.Series(dmr_annot.X.max(axis=0).todense().A1 * mask_quantile_to_max, index=dmr_annot.var_names)

In [10]:
dmr_annot = dmr_annot[use_dmrs, :].copy()
dmr_annot

AnnData object with n_obs × n_vars = 4059 × 719 
    obs: 'chrom', 'start', 'end'

## Background Hits

In [11]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [12]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [13]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, dmr_annot.var_names].copy()
background_motif_hits

AnnData object with n_obs × n_vars = 346210 × 719 
    obs: 'chrom', 'start', 'end'

## Redo motif score filter

In [14]:
# only keep value larger than the cutoff for each motif
dmr_annot.X = dmr_annot.X.multiply(
    (dmr_annot.X >
     motif_cutoff[dmr_annot.var_names].values[None, :]).astype(int)).tocsr()

In [15]:
# only keep value larger than the cutoff for each motif
background_motif_hits.X = background_motif_hits.X.multiply(
    (background_motif_hits.X >
     motif_cutoff[background_motif_hits.var_names].values[None, :]).astype(int)).tocsr()

## Motif hits contingency table

In [16]:
motif_ids = dmr_annot.var_names

# calculate motif occurence, not considering hits here
pos = (dmr_annot[:, motif_ids].X > 0).sum(axis=0)
pos_total = dmr_annot.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [17]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [18]:
results = {}
with ProcessPoolExecutor(40) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

In [19]:
motif_enrich_df['GeneCluster'] = gene_cluster
motif_enrich_df['DMRType'] = 'Hypo'

In [20]:
motif_enrich_df = motif_enrich_df[motif_enrich_df['oddsratio'] > 1].copy()

## Add gene info

In [21]:
motif_gene_anno = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/MotifClustering/JASPAR2020_CORE_vertebrates_non-redundant.mouse_genes.with_motif_group.199.csv', 
    index_col=0
)
motif_gene_anno.head()

,motif_name,motif_genes,gene_ids,gene_names,motif_group
motif_uid,,,,,
MA0006.1,Ahr::Arnt,"Ahr,Arnt","ENSMUSG00000019256.17,ENSMUSG00000015522.18","Ahr,Arnt",MotifGroup178
MA0854.1,Alx1,Alx1,ENSMUSG00000036602.14,Alx1,MotifGroup3
MA0634.1,ALX3,ALX3,ENSMUSG00000014603.3,Alx3,MotifGroup3
MA0853.1,Alx4,Alx4,ENSMUSG00000040310.12,Alx4,MotifGroup3
MA0007.3,Ar,Ar,ENSMUSG00000046532.8,Ar,MotifGroup32


In [22]:
motif_enrich_df = pd.concat([motif_enrich_df, motif_gene_anno.reindex(motif_enrich_df.index)], axis=1)

In [23]:
motif_enrich_df.to_msgpack(output_dir / f'{gene_cluster}.Hypo.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [24]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

16

In [25]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,GeneCluster,DMRType,motif_name,motif_genes,gene_ids,gene_names,motif_group
MA0027.2,1.659129,4.777631e-14,4.907309e-12,11.309157,273.0,3786.0,14420.0,331790.0,0.067258,0.041651,13,Hypo,EN1,EN1,ENSMUSG00000058665.8,En1,MotifGroup3
MA0068.2,1.612506,1.092885e-08,2.806372e-07,6.551855,169.0,3890.0,9083.0,337127.0,0.041636,0.026236,13,Hypo,PAX4,PAX4,ENSMUSG00000029706.15,Pax4,MotifGroup3
MA0075.3,1.713127,9.435278e-18,2.261322e-15,14.645638,320.0,3739.0,16473.0,329737.0,0.078837,0.047581,13,Hypo,PRRX2,PRRX2,ENSMUSG00000039476.13,Prrx2,MotifGroup3
MA0612.2,1.662624,2.883095e-08,5.602555e-07,6.251614,141.0,3918.0,7335.0,338875.0,0.034738,0.021187,13,Hypo,EMX1,EMX1,ENSMUSG00000033726.8,Emx1,MotifGroup3
MA0618.1,1.603612,2.542954e-15,4.570961e-13,12.339993,348.0,3711.0,19127.0,327083.0,0.085735,0.055247,13,Hypo,LBX1,LBX1,ENSMUSG00000025216.9,Lbx1,MotifGroup3
MA0722.1,1.624322,4.756515e-14,4.907309e-12,11.309157,297.0,3762.0,16047.0,330163.0,0.073171,0.046350,13,Hypo,VAX1,VAX1,ENSMUSG00000006270.7,Vax1,MotifGroup3
MA0725.1,1.713127,9.435278e-18,2.261322e-15,14.645638,320.0,3739.0,16473.0,329737.0,0.078837,0.047581,13,Hypo,VSX1,VSX1,ENSMUSG00000033080.9,Vsx1,MotifGroup3
MA0726.1,1.713127,9.435278e-18,2.261322e-15,14.645638,320.0,3739.0,16473.0,329737.0,0.078837,0.047581,13,Hypo,VSX2,VSX2,ENSMUSG00000021239.12,Vsx2,MotifGroup3
MA0886.1,1.712155,2.512396e-13,2.007126e-11,10.697425,228.0,3831.0,11630.0,334580.0,0.056171,0.033592,13,Hypo,EMX2,EMX2,ENSMUSG00000043969.4,Emx2,MotifGroup3
MA0889.1,1.680219,5.994613e-14,5.387659e-12,11.268600,258.0,3801.0,13443.0,332767.0,0.063562,0.038829,13,Hypo,GBX1,GBX1,ENSMUSG00000067724.5,Gbx1,MotifGroup3
